# Projet Machine Learnig

## Dataset UCI Support2 

### Membre de groupe
*    Abdeldjalil SMAHI : abdeldjalil.smahi@ens.uvsq.fr
*    Oussama BOUAKAZ : oussama.bouakaz@ens.uvsq.fr

## Importation des bibliothèques

N'exécutez pas ces deux cellulle si vous avez déj Xgboost et scikit-plot

In [ ]:
!pip install xgboost

In [ ]:
!pip install scikit-plot

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OrdinalEncoder
from collections import Counter
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import GridSearchCV
import scikitplot as skplt
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import jaccard_score
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist


In [ ]:
import multiprocessing
import psutil

print("Nombre de CPU logiques :", multiprocessing.cpu_count())
print("Nombre de CPU physiques :", psutil.cpu_count(logical=False))

### Configuration des affichages pandas

In [ ]:
# Pour afficher toutes les colonnes
pd.set_option('display.max_columns', None)

# Pour afficher toutes les lignes
pd.set_option('display.max_rows', None)

## Importation des données

Nous avons mis le dataset dans le repo github et puis l'importer directement du repo.


Référence du dataset :


*   [UCI Support2 Dataset](https://archive.ics.uci.edu/dataset/880/support2)
*   [Description du dataset](https://hbiostat.org/data/repo/supportdesc)



**SUPPORT 2 dataset** est une collection de données publié sur : [UCI Support2 Dataset](https://archive.ics.uci.edu/dataset/880/support2). Collecté entre *1989 – 1991 et 1992-1994* de **5 centres médicaux localisés aux USA**. Cette collection correspand aux données du **9105 patients** gravement malades telque chaque ligne correspond à un dossier du patient hospitalisé qui répond aux critères d’inclusion et d’éxclusion pour neuf catégories de maladies :
*   insuffisance respiratoire aiguë
*   maladie pulmonaire obstructive chronique
*   insuffisance cardiaque congestive
*   maladie du foie
*   coma
*   cancer du côlon
*   cancer du poumon
*   défaillance de plusieurs organes avec malignité
*   défaillance de plusieurs organes avec sepsis1



Le dataset est composé de **45** colonnes (attributs) dont 3 peuvent être utilisées comme targets (y’en a d’autres) et le **42** qui restent comme caractéristiques.
Le code suivant affiche une sous ensemble du dataset (50 premiers ligne) :

In [ ]:
url = "https://raw.githubusercontent.com/abdeldjalilSmahi/ML_Project/main/support2_dataset.csv"

# Importation du dataset depuis l'URL
data = pd.read_csv(url)

# Affichage des 50 premières lignes de notre dataset
data.head(50)

## Analyse Exploratoire des Données (EDA)

### Déscription du dataset

Alors, comme indiqué précedement, le dataset est composé de 9105 ligne et 45 colonnes comme l'indique le code ci-dessous :

In [ ]:
df = data.copy()

In [ ]:
df.shape

Le code suivant donne des informations sur le dataset, nous allons expliqué par la suite chaque attribut en se basant sur la discription du site **UCI**.

In [ ]:
df.info()

**Explication de chaque colonne**

*   **age:**  Cette variable, indique l'age du patient en *année* au moment de son entrée dans l'étude. Est une variable numérique et représente **une caractéristique (feature)** .
*   **sex:** Sexe du patient. est une variable catégorielle qui peut prendre l'une des deux valeurs {male, female}.
*   **dzclass:** La catégorie de la maladie du patient, une caractéristique catégorielle qui peut prendre l'une de ces valeurs : {ARF/MOSF", "COPD/CHF/Cirrhose", "Cancer", "Coma"}.  telque *ARF/MOSF* indique l'insuffisance rénale aigue/syndrome de dysfonctionnement multiviscéral, *COPD/CHF/Cirrhose* indique que le patient atteint une Bronchopneumopathie obstructive chronique (BPCO)/insuffisance cardiaque congestive (ICC)/cirrhose du foie.
*   **dzgroup:** C'est la sous catégorie de la maladie du patient, est une caracatérisitque catégorielle qui prend les valeurs : {ARF/MOSF w/Sepsis, CHF, COPD, Cirrhosis, Colon Cancer, Coma, Lung Cancer, MOSF w/Malig}: ARF/MOSF w/Sepsis indique : Insuffisance rénale aiguë (IRA)/syndrome de dysfonctionnement multiviscéral (MODS) avec sepsis. CHF indique : Insuffisance cardiaque congestive (ICC). COPD indique : Bronchopneumopathie obstructive chronique (BPCO). Cirrhosis inidique : Cirrhose du foie. Colon Cancer : Cancer du côlon et du rectum. Coma inidique : État de conscience altéré profond et prolongé. Lung Cancer indique : Cancer du poumon. MOSF w/Malig indique : Syndrome de dysfonctionnement multiviscéral (MODS) avec malignité.
*   **num.co:**   Nombre de maladie simultannées ou comorbidités qui veut dire la présence d'une maladie supplémentaire qui coexiste avec la maladie principlae du patient. cette varibale prend des valeurs numérique, la plus élevée indique **un état de chances de survie plus mauvais**

*   **edu:** nombre d'année d'études.
*   **income** :  Revenu du patient annulle en k\$. Cette variable est importante, elle nous donne une idée sur la possiblité de patient de rester hispotalisé, les patients ayant un niveau de revenu plus élevé sont moins susceptibles de décéder par rapport aux patients ayant un niveau de revenu plus faible.
*   **scoma:** score de scoma du jour 3 basé sur l'échelle de Glasgow, la valeur est prédite par un modèle et est utilisé pour évaluer le niveau du conscience d'un patient en prenant en conisdération 3 critères l'ouverture des yeux, la réponse verbale et la réponse motrice. cette variable joue un role important car les patients ayant un score de coma plus bas sont les plus susceptibles de décéder.
*   **charges:**  frais d’hospitalisation, elle indique les frais d’hospitalisation du patient.  
*   **totcost :**  cout total du ratio couts/frais, cette variable indique le cout total du ration couts/frais du patient qui indique l’éfficacité des soins fournis au patient.
*   **totmcst :** cette variable indique le cout total micro du patient, c’est cout estimatif total des soins fournis aux patient.
*   **avtisst :** Score TISS moyen du patient sur les jours 3 à 25 de son hospitalisation, elle permet d’évaleur l’intensité des soins fournis au patient.
*   **race :** La race du patient, variable catégorielle, qui peut prendre des valeurs {asian, black, hispanic, missing, other, white}.
*    **sps** :  Score de physilogie du jour 3 du patient ele permet d’évaluer la gravité de sa maladie. cette valeur est prédite par un autre modèle.
*    **aps** : Score de physiologie APCHE III du jour 3 sans coma, byn , uout. Elle permet d’évaluer la gravité de maladie d’un patient en USI ou en USIM.
*   **surv2m :** les valeurs de cette colonne sont estimé par un autre modèle, elles indiquent l’estimation de la survie du patient à 2 mois au jour 3 de son hospitalisation. [](url) dans la documentation recommande d’ignorer cette colonne au moment de l’entrainement de notre modèle pour ne pas influencer son apprentissage.
*    **surv6m :**  Comme la variable précedente, cette variable est prédite par un modèle elle indique l’estimation de la survie à 6 mois du patient au jour 3.
*   **hday** : La variable hday, indique le jours d’hospitalisation auquel le patient est entré dans l’étude.
*   **diabetes** :  cette variable indique si le patient est diabétique ou pas afin d’étudier l’impact de cette maladie sur le pronostic du patient. 0 si non, 1 s’il presente le diabète.
*    **dementia:** indique si le patient présente de deux ou plusieurs maladies. 0 si non, 1 si oui .
*  **ca :** cette variable indique si le patient a un cancer ou pas et s’il est propagé à d’autres partie du corps. Elle prend les valeurs suivantes {no, yes, metastatic}.  
*   **prg2m :** represente l’estimation du médecin du taux de survie du patient à 2 mois en pourcentage (entre 0 et 1)
*   **prg6m :** represente l’estimation du médecin du taux de survie du patient à 6 mois en pourcentage (entre 0 et 1)
*   **dnr :** indique si le patient a ou non un ordre de ne pas réanimer. Elle prend les valeurs suivantes : {no dnr, dnr after sadm, dnr before sadm}
*   **dnrday:** le jour de l’ordre dnr du patient <0 si l’ordre a été donné avant l étude.
*   **meanbp :** cette variable indique la pression artérielle mpyenne du patient mesurée au jour 3.  Une valeur elevée  peut augmenter le risque de maladies cardiovasculaires.
*   **wblc :** idnique le nombre de globules (en milliers) mesuré au jour 3.
*  **hrt** : cette variable indique la fréquence cardiaque du patient mesurée au 3 eme jour, mesuré en nombre de battements par seconde.
* **resp :** indique la fréquence réspiratoire du patient. mesuré en nombre de respirations par minute.
*    **temp** :  cette variable indique la temparture corporelle en °C du patient.
*   **pafi:** est une variable numérique qui indique le rapport PaO2/fiO2 (pression partielle d'oxygène dans le sang artériel par oxygène inspiré), un rapport normal devrait etre sup à 300

### Distribution des types de données

Dans la problèmatique supervisé, et comme indiqué dans le catalogue du dataset qu'il existe 3 targets dans ce dataset. Nous allons considérer que la colonne **death** comme colonne target et faire une classification binaire la-dessus.

In [ ]:
df = df.drop(['hospdead', 'sfdm2'], axis = 1)

Le graphique circulaire ci-dessous, indique la distribution des types de colonnes dans le dataset

In [ ]:
df.dtypes.value_counts().plot.pie()

In [ ]:
df.dtypes.value_counts()

Plus précisément, nous avons : 
Types de variables :     
*    **quantitatives** : 37
*    **qualitatives** : 8

### Analyse univariée

#### Distribution des classes 

In [ ]:
class_counts = df['death'].value_counts()
print(class_counts)

In [ ]:
unique, counts = np.unique(df[['death']], return_counts=True)
plt.title('Distribution des classes')
sns.barplot(x=unique, y=counts)

Nous déduisons que dans la problématique supervisé, nous sommes face à un problème de classification binaire dans lequel les classes ne sont pas équi-distribué, nous devons prendre ça en considération par la suite.

In [ ]:
df['death'].value_counts(normalize = True)

Parmi les 9105 patients de cette étude, 68% entre eux ont décédé, et 32% entre eux ont survécu.

##### Map des valeures nulles

In [ ]:
plt.figure(figsize=(20,10))
sns.heatmap(df.isna())

Y'en a pas beaucoup de champs blancs, ce qui veut dire qu'il y en a très peu de valeurs manquantes. Le code suivant, va nous aider à calculer la proportion des valeurs manquantes et les trier.

In [ ]:
(df.isna().sum()/df.shape[0]).sort_values(ascending=True)

##### Analyse de chaque Variable

Séparation des Variables Numériques et Catégoriques

In [ ]:
colonnes_numeriques = df.select_dtypes(include=['int64', 'float64'])
colonnes_categoriques = df.select_dtypes(include=['object'])

On analyse les variables numériques d'un ensemble de données. Il fournit des statistiques essentielles telles que la moyenne, l'écart type et les quartiles, montrant les tendances centrales et la dispersion des données. Les graphiques produits (histogramme et boîte à moustaches) offrent des vues visuelles de la distribution et des caractéristiques clés.
Voici l'intrepretation de la fonction describe ():
*  Count : Le nombre total d'éléments dans la variable.
*  Mean : La moyenne, c'est-à-dire la valeur centrale des données.
*  Std : L'écart type, indiquant à quel point les valeurs sont dispersées autour de la moyenne.
*  Min : La plus petite valeur dans la variable.
*  25% : Le premier quartile, représentant le point en-dessous duquel se trouvent 25% des données.
*  50% : La médiane, divisant les données en deux moitiés égales.
*  75% : Le troisième quartile, en-dessous duquel se situent 75% des données.
*  Max : La plus grande valeur dans la variable.


In [ ]:
colonnes_numeriques.describe()

In [ ]:
# Boucle sur chaque colonne numérique du dataset
for column in colonnes_numeriques.columns:

    summary_statistics = colonnes_numeriques[column].describe()
    print(f"Résumé statistique pour la variable '{column}':")
    print(summary_statistics)
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(6, 4))

    # Histogramme de la colonne
    axes[0].hist(colonnes_numeriques[column], bins=20, color='skyblue', edgecolor='black')
    axes[0].set_title(f'Histogramme pour {column}')
    axes[0].set_xlabel(column)
    axes[0].set_ylabel('Fréquence')

    # Boîte à moustaches (boxplot) de la colonne
    sns.boxplot(x=colonnes_numeriques[column], color='skyblue', ax=axes[1])
    axes[1].set_title(f'Boîte à moustaches pour {column}')
    axes[1].set_xlabel(column)

    # Ajuster l'espacement entre les sous-graphes pour éviter les chevauchements
    plt.tight_layout()

    # Afficher les graphes
    plt.show()
    missing_values = colonnes_numeriques[column].isnull().sum()
    print(f"\n Nombre de valeurs manquantes pour '{column}': {missing_values}\n")

Nous appliquons maintenant la même méthode d'analyse aux variables catégoriques,on affiche le résumé statistique généré par describe(), montrant des statistiques clés comme le nombre total d'occurrences (count), la catégorie la plus fréquente (top) et son nombre d'occurrences (freq).



In [ ]:
colonnes_categoriques.describe()

In [ ]:
for col in df.select_dtypes('object'):
    print(f'{col :-<50} {df[col].unique()}')

In [ ]:
for column in colonnes_categoriques.columns:
    print(f"Résumé statistique pour la variable '{column}':")
    print(colonnes_categoriques[column].describe())

    fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(8, 4))


    sns.countplot(x=column, data=colonnes_categoriques)
    axes.set_title(f'Diagramme en barres pour {column}')
    axes.set_xlabel(column)
    axes.set_ylabel('Fréquence')
    # Ajustement l'espacement entre les sous-graphes pour éviter les chevauchements
    plt.tight_layout()

    # Affichage de graphe
    plt.show()
     # Affichage des valeurs manquantes pour la colonne
    missing_values = colonnes_categoriques[column].isnull().sum()
    print(f"\nNombre de valeurs manquantes pour '{column}': {missing_values}\n")

Pi plot pour les variables categoriques

In [ ]:
for col in colonnes_categoriques :
    plt.figure()
    df[col].value_counts().plot.pie(autopct="%.2f%%")
         # Affichage des valeurs manquantes pour la colonne
    missing_values = data[col].isnull().sum()
    print(f"\nNombre de valeurs manquantes pour '{col}': {missing_values}\n")

### Analyse bivariée

#### Relation entre les variables et la target *death*

Pour faciliter cette étude, on a deux groupe de patients, ceux qui ont décédé, et ceux qui ont survecu.

On les sépare

In [ ]:
death_patients = df[df['death']== 1]
survived_patients = df[df['death']== 0]

Nous allons montrer avec des graphes la relation de nos variables avec l'aatribut target

In [ ]:
for col in colonnes_numeriques:
    plt.figure()
    sns.distplot(death_patients[col], label='death')
    sns.distplot(survived_patients[col], label='survive')
    plt.legend()

In [ ]:
for col in colonnes_categoriques :
    plt.figure()
    sns.heatmap(pd.crosstab(df['death'], df[col]), annot=True, fmt='d')

En analysant les résultats ci-dessus:
*    Le nombre de décés des males est supérieurs que les females.
*    ceux qui sont atteint par *ARF/MOSF w/Sepsis* Insuffisance rénale aiguë (IRA)/syndrome de dysfonctionnement multiviscéral (MOFS) avec sepsis sont plus susceptible de mourir<
*   Ceux aui touche ;oins de 11K $ par an sont plus susceptible de mourir, est cela du peut etre de l'incapcité de payer les charges hospitalières et les soins médicaux.
*   Ceux qui ont un cancer metastatic, càd qui s'est propagé dans le corps sont plus susceptible de mourir.

#### La matrice de corrélation

In [ ]:
# Affichage de la matrice de corrélation
plt.figure(figsize=(48, 48))
sns.heatmap(colonnes_numeriques.corr(), annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Matrice de Corrélation entre Toutes les Variables')
plt.show()

*    La matrice de corrélation nous indique que **adlsc** et **adls** sont fortement corrélés, et c'est normal car les deux pratiquement represente la meme information, l'indice des activités quotidiennes (ADL) du patient.
*    **surv6m** et **surv2m** sont aussi fortement corrélé car est une estimation à vivre de 2 mois et l'autre à 6 mois, et le fait que la possibilité que le patient meurt dans 2 mois donc il sera décédé d'ici 6 mois, on rappelle que ces valeurs sont prédites par un modèle. 
*    **prg2m** et **prg6m** sont aussi fortement corrélé, comme les précédentes sauf que c'est une estimation des medecins.

## Prétraitement / Préparation des données

Dans le cadre de notre problématique supervisé, les colonnes fortement corrélées peuvent nuire à la performance du modèle. Il est donc recommandé de les supprimer pour éviter le surapprentissage. Si deux variables sont fortement corrélées, cela signifie qu'elles varient de manière similaire. Dans le cadre de l'apprentissage supervisé, cela signifie que le modèle peut apprendre à prédire la variable cible en se basant sur l'une des variables corrélées. Cela peut entraîner le surapprentissage, ce qui signifie que le modèle s'adapte trop aux données d'entraînement et ne parvient pas à généraliser aux données de test.

Nous avons dit que **adls** et **adlsc** sont fortement corrélé. Nous allons supprimer **adls** car il possède des valeurs manquantes contrairement à **adlsc** . 

In [ ]:
df = df.drop(['adls'], axis=1)

De plus, suite à notre lecture de la littérature sur le dataset [[Description du dataset](https://hbiostat.org/data/repo/supportdesc)] , il est recommandé, dans le cadre de la création d'un modèle, de ne pas inclure les colonnes **aps**, **sps**, **surv2m** et **surv6m** en tant que prédicteurs. Il est également probablement préférable de ne pas utiliser les colonnes **prg2m**, **prg6m**, **dnr**, **dnrday**. Par ailleurs, **prg2m** et deja fortement corrélé avec **prg6m**, et **dnrday** est fortement corrélé avec **totcst** et **totmcst** en plus **totcst** et **totmcst** sont fortement corrélé avec **charge** , raison pour laquelle nous supprimons les colonnes **prg2m**, **prg6m**, **dnrday**, **totcst** et **totmcst** [matrice de corrélation ci dessus]

In [ ]:
df = df.drop(['aps', 'sps', 'surv2m', 'surv6m'], axis=1)

In [ ]:
df = df.drop(['prg2m', 'prg6m', 'dnrday', 'totcst', 'totmcst'], axis=1)

Concernant les colonnes **dzclass** et **dzgroup**, deux variables catégorielles implicitement corrélées, l'examen du descriptif du dataset révèle que **dzgroup** représente la sous-catégorie de la maladie, tandis que **dzclass** représente la catégorie globale de la maladie. En d'autres termes, les deux variables font référence à la même information, mais **dzgroup** offre un niveau de détail plus élevé que **dzclass**. Par conséquent, nous conservons **dzgroup** à la place de **dzclass**.

In [ ]:
df = df.drop(['dzclass'], axis=1)

### Remplissage des champs nulls

Maintenant, c'est le moment de s'occuper avec les valeurs manquantes.

Y'en a plusieurs strategies pour traiter ce genre de situation des valeurs null:


1.   **Suppression des lignes contenant les valeurs null** : Supprimer ces ligne, cependant cette stratégie n'est pas trop admirable surtout si le nombre de ces tuples est considérable par rapport à la taille de dataset et ça peut entraîner une perte d'informations importantes si les valeurs manquantes sont réparties dans plusieurs colonnes ou lignes.
2.   **Remplir par des statistiques** : Remplaces les valeurs null par des statistiques telles que la moyenne, la médiane ou par la valeur la plus fréquente (mode) "généralement pour les attributs de type numérique on opte par la stratégie de la moyenne mais tout depend". cette stratégie est simple et rapide à mettre en oeuvre mais elle peut générer un bruit sur la dataset que nous voyons un épée à double tranchant pour éviter le surapprentissage.



Nous allons opter pour la deuxième strategie à savoir le remplissage par des statisiques.
Nous réccupérons les colonnes qui possèdent des valeurs manquantes dans une variables : 

In [ ]:
null_columns = df.columns[df.isnull().any()]
null_columns

Nous utilisons le SimpleImputer en lui indiquant la stratégie de remplissage

In [ ]:
edu_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
income_imputer = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
scoma_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
charges_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
avtisst_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
race_imputer = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
dnr_imputer = SimpleImputer(missing_values= np.nan, strategy='most_frequent')
meanbp_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
wblc_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
hrt_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
resp_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
temp_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
pafi_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
alb_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
bili_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
crea_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
sod_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
ph_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
glucose_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
bun_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
urine_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')
adlp_imputer = SimpleImputer(missing_values= np.nan, strategy='mean')

In [ ]:
ct= ColumnTransformer(transformers=[
      ('edu_imputer', edu_imputer, ['edu']),
      ('income_imputer', income_imputer, ['income']),
      ('scoma_imputer', scoma_imputer, ['scoma']),
      ('charges_imputer', charges_imputer, ['charges']),
      ('avtisst_imputer', avtisst_imputer, ['avtisst']),
      ('race_imputer', race_imputer, ['race']),
      ('dnr_imputer', dnr_imputer, ['dnr']),
      ('meanbp_imputer', meanbp_imputer, ['meanbp']),
      ('wblc_imputer', wblc_imputer, ['wblc']),
      ('hrt_imputer', hrt_imputer, ['hrt']),
      ('resp_imputer', resp_imputer, ['resp']),
      ('temp_imputer', temp_imputer, ['temp']),
      ('pafi_imputer', pafi_imputer, ['pafi']),
      ('alb_imputer', alb_imputer, ['alb']),
      ('bili_imputer', bili_imputer, ['bili']),
      ('crea_imputer', crea_imputer, ['crea']),
      ('sod_imputer', sod_imputer, ['sod']),
      ('ph_imputer', ph_imputer, ['ph']),
      ('glucose_imputer', glucose_imputer, ['glucose']),
      ('bun_imputer', bun_imputer, ['bun']),
      ('urine_imputer', urine_imputer, ['urine']),
      ('adlp_imputer', adlp_imputer, ['adlp'])
])

In [ ]:
# On enregistre  les types de données d'origine : 
original_dtypes = df.dtypes
df_transformed = pd.DataFrame(ct.fit_transform(df), columns=['edu', 'income', 'scoma', 'charges', 'avtisst', 'race','dnr',
       'meanbp', 'wblc', 'hrt', 'resp', 'temp', 'pafi', 'alb', 'bili', 'crea',
       'sod', 'ph', 'glucose', 'bun', 'urine', 'adlp'])
for col in df_transformed.columns:
    df_transformed[col] = df_transformed[col].astype(original_dtypes[col])

df[['edu', 'income', 'scoma', 'charges', 'avtisst', 'race','dnr',
       'meanbp', 'wblc', 'hrt', 'resp', 'temp', 'pafi', 'alb', 'bili', 'crea',
       'sod', 'ph', 'glucose', 'bun', 'urine', 'adlp']] = df_transformed

In [ ]:
df.info()

In [ ]:
colonnes_numeriques = df.select_dtypes(include=['int64', 'float64'])
colonnes_categoriques = df.select_dtypes(include=['object'])

### Encodage des variables catégoriques

Nous allons encoder nos variables categoriques selon la nature de chacune si la variable posséde un ordre important entre ses valeurs nous utilisons ***ordinalEncoding*** si elle ne posséde pas un ordre entre ses valeurs nous optons pour le **oneHotEncoding**

In [ ]:
col_cats_name= list(colonnes_categoriques.columns)

In [ ]:
col_cats_name

Comme nous voyons dans le dataset, les valeurs da la colonne (incom) *revenue des patients* ont un ordre entre eux.

In [ ]:
col_cats_name.pop(2)

In [ ]:
df['income'] = df['income'].replace({
    'under $11k': 0,
    '$11-$25k': 1,
    '$25-$50k': 2,
    '>$50k': 3
})

Comme la variable sex prend que deux valeurs différentes, nous nous permettons de l'encoder en utilisant **OrdinalEncoding** . 

In [ ]:
encoder = OrdinalEncoder()
df[['sex']] = encoder.fit_transform(df[['sex']]).astype('int')

In [ ]:
col_cats_name.pop(0)

In [ ]:
df = pd.get_dummies(df, columns=col_cats_name)

In [ ]:
df.head(50)

In [ ]:
plt.figure(figsize=(48, 48))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm', linewidths=.5)
plt.title('Matrice de Corrélation entre Toutes les Variables')
plt.show()

En analysant la matrice y'a pas de correlation entre nos variables sauf pour les variables qui appartiennet à la meme colonne de base avant l'utilisation de onehoteEncoding , ce qui est logique .

## Features selection

La sélection des caractéristiques est un processus itératif qui vise à identifier un sous-ensemble de variables pertinentes à partir des caractéristiques du dataset. Dans notre cas nous avons procédé de plusieurs méthodes de selection des caractéristiques afin de réduire le nombre de features: 

**Analyse des Composantes Principales (PCA)**:Nous allons nous servir de cette méthode en prenant les coefficients des features qui représentent des valeurs numériques qui indiquent l'importance d'une variable dans la projection, pour ce faire nous allons considérer les deux premières composantes importantes et nous prenons les variables qui possèdent les coefficients les plus élevés.

**Sélection de K-meilleures Variables** : Cette méthode consiste à choisir un nombre \textbf{K} de features à prendre en considération, elle selectionne les K caractéristiques les plus informatives dans le dataset selon plusieurs critères dans notre cas nous avons choisi **la variance**.

**Sélection en se basant sur un Classifier** : Nous entraînons un Classifier RandomForest sur nos donnés qui est capable d'identifier l'importance de chaque caractéristique pour prédire la cible, ensuite, nous utilisant **SelectFromModel** afin de selectionner les caractéristiques les plus pertinentes.

A la fin de ces trois méthodes, nous faisons **une intersections** des caractéristiques afin d'avoir un ensemble de features plus restreints.


Ici on fait une copie de notre dataset.

In [ ]:
dfs = df.copy()

In [ ]:
dfs_y = dfs.death
dfs_x = dfs.drop(['death'], axis=1)

In [ ]:
dfs_x.info()

Nous allons dans un premier temps choisir les variables qui devraient être mises à l'échelle qui vont etre la variable ou y a des escarts entres leurs valeurs : 

In [ ]:
dfs.describe()

In [ ]:
cols_to_scale = ['age', 'edu', 'num.co', 'scoma', 'charges', 'avtisst','hday', 'meanbp', 'wblc', 'hrt', 'resp', 'temp', 'pafi', 'alb', 'bili', 'crea', 'sod', 'ph', 'glucose', 'bun', 'urine', 'adlp', 'adlsc']

In [ ]:
sc = StandardScaler()
dfs_x[cols_to_scale] =  sc.fit_transform(dfs_x[cols_to_scale].values)

### PCA (analyse des composantes principales)

Nous allons utiliser pca comme une premiére approche qui va nous aider à selectionner nos variables pertinentes et à reduire les dimensions . Le code ci-dessous transforme les variables originales en composantes principales. Le code utilise la bibliothèque scikit-learn pour effectuer la PCA, puis affiche le pourcentage de variance expliquée par chaque composante principale dans un graphique à barres appelé "Scree Plot". Ce graphique aide à déterminer combien de composantes principales doivent être conservées pour représenter efficacement les données. 

In [ ]:
pca = PCA()
pca.fit_transform(dfs_x)
print(np.sum(pca.explained_variance_ratio_))
per_var = np.round(pca.explained_variance_ratio_,decimals=1)
labels = ['PC' + str(x) for x in range(1, len(per_var)+1)]
plt.subplots(nrows=1, ncols=1, figsize=(12, 12))
plt.bar(x=range(1,len(pca.explained_variance_ratio_)+1), height=pca.explained_variance_ratio_, tick_label=labels)
plt.ylabel('Percentage of Explained Variance')
plt.xlabel('Principal Component')
plt.title('Scree Plot')
plt.show()

pca.components_[i] contient les coefs attribués à nos variables pour la i éme composante , dans notre cas on va prendre les deux premiéres composantes.
Ce code extrait et trie les composantes principales obtenues à partir de l'analyse en composantes principales (PCA) pour chaque colonne du DataFrame `dfs_x`. Les paires clé-valeur triées sont stockées dans une liste appelée `list_components`.

In [ ]:
list_components = []
for i in range(0,2):
    result_dict = {}
    for key, value in zip(dfs_x.columns, np.absolute(pca.components_[i])):
        result_dict[key] = value
    list_component = sorted(result_dict.items(), key=lambda x: x[1])
    list_components.append(list_component)


Le code ci-dessous compte le nombre d'occurrences des clés (noms de colonnes) dans les listes triées de paires clé-valeur stockées dans `list_components`. Il sélectionne les 33 clés les plus fréquentes parmi toutes les listes triées et les affiche. En résumé, il identifie les 33 noms de colonnes les plus importants à partir des composantes principales extraites par l'analyse en composantes principales (PCA).

In [ ]:
key_counts = Counter()
for data_list in list_components:
    sorted_data = sorted(data_list, key=lambda x: x[1], reverse=True)[:33]
    key_counts.update(dict(sorted_data).keys())

top_keys = [key for key, count in key_counts.most_common(33)]

print(top_keys)


### Selection à partir du RandomForest

Ce code utilise un modèle de forêt aléatoire pour sélectionner les caractéristiques les plus importantes à partir de nos données (dfs_x dfs_y). Il imprime ensuite les noms de ces caractéristiques sélectionnées

In [ ]:
rfc = RandomForestClassifier()

selector = SelectFromModel(rfc)
selector.fit(dfs_x, dfs_y)

feature_indices = selector.get_support()

feature_names = [dfs_x.columns[i] for i in range(len(dfs_x.columns)) if feature_indices[i]]

print(feature_names)

### Selection à partir du K-Best

Ce code applique la méthode de sélection de fonctionnalités SelectKBest avec k=33, ce qui signifie qu'il sélectionne les 33 meilleures caractéristiques en se basant sur leur score. Il utilise ensuite les indices des caractéristiques sélectionnées pour extraire les noms correspondants des colonnes du DataFrame dfs_x. Enfin, il imprime les noms des caractéristiques sélectionnées.

In [ ]:
selector = SelectKBest( k=33)
selector.fit(dfs_x.values, dfs_y.values)
feature_indices = selector.get_support()

feature_names_ = [dfs_x.columns[i] for i in range(len(dfs_x.columns)) if feature_indices[i]]


print(feature_names_)

A fin de choisir les variables les plus pertinentes à partir de 3 methodes mentionnées ci-dessus , si une varibale presentes dans 2 ou 3 vecteurs on la prends.

In [ ]:
cols = list(dfs_x.columns) 
cols_to_select = []
for col in cols : 
    v1 = col in top_keys
    v2 = col in feature_names_
    v3 = col in feature_names
    listin = [v1, v2, v3]
    if listin.count(True) >= 2:
        cols_to_select.append(col)
print(cols_to_select)

Après avoir executé les trois méthodes de selections de caractéristiques, nous avons décidé, de prendre les colonnes suivantes pour passer à l'étape de conception des modèles. 

In [ ]:
cols_to_select.extend(['death'])

En examinant le descriptif du dataset, nous remarquons également que le niveau d'étude du patient (**edu**) ne semble pas être une variable significative pour prédire son pronostic de survie ou de décès, donc il est nécessaire de ne pas considérer la variable edu.

In [ ]:
cols_to_select.pop(2)

On imprime les variables qu'on va travailler avec

In [ ]:
cols_to_select

In [ ]:
dfs = dfs[cols_to_select]

In [ ]:
dfs.info()

## Augmentation des données

Comme indiqué précédemment, notre ensemble de données présente un déséquilibre notable.

In [ ]:
class_counts = dfs['death'].value_counts()
print(class_counts)

In [ ]:
class_1_count, class_0_count = dfs['death'].value_counts()
class_1_count, class_0_count

Nous allons effectue un sur-échantillonnage de la classe  (`death` égal à 1) dans le DataFrame `dfs` en créant un échantillon aléatoire de la classe majoritaire (`death` égal à 0) de la même taille que la classe minoritaire. Ensuite, il concatène cet échantillon suréchantillonné avec la classe minoritaire pour obtenir un ensemble de données équilibré (`dfs_balanced`) en termes de classes. Enfin, le code imprime le décompte des classes dans le nouvel ensemble de données pour vérifier qu'il est équilibré.

In [ ]:

class_0 = dfs[dfs['death'] == 0]
class_0_oversampled = class_0.sample(class_1_count, replace=True)

# Concatenatenation de la classe rééchantillée :  '>50K' et la classe dominante'<=50K' afin d'obtenir la dataset balanced.
dfs_balanced = pd.concat([dfs[dfs['death'] == 1], class_0_oversampled], axis=0)

# Vérification
class_counts = dfs_balanced['death'].value_counts()
print(class_counts)

### Standarisation des données

In [ ]:
dfs = dfs_balanced

In [ ]:
dfs.info()

In [ ]:
y = dfs.pop('death').values

In [ ]:
cols_to_scale = list(set(cols_to_scale).intersection(set(list(dfs.columns)))) 
cols_to_scale

In [ ]:
X = dfs

On divise notre ensemble de données et etiquette en deux groupe (train et test)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25,
                                                    random_state = 1,stratify=y)

Création du transformateur de colonnes avec StandardScaler pour les colonnes à mettre à l'échelle

In [ ]:

ct = ColumnTransformer([
    ('scale', StandardScaler(), cols_to_scale)
], remainder='passthrough')


X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

## Apprentissage supervisé

On a exécuté le GridSearch en utilisant Jupyter Notebook du cluster Paris Saclay  : https://jupyterhub.ijclab.in2p3.fr/ qui utilise : 


*   Nombre de CPU logiques : 240
*   Nombre de CPU physiques : 240

Nous allons opter pour des paramètres initiaux dans nos modèles supervisés, afin de mener une recherche exhaustive sur la grille (grid search) et d'obtenir les meilleurs parametres pour chaque modèle.

### Support Vector Machine (SVM)


1. **`C`** : Ce paramètre régularise le modèle. Une valeur plus élevée de `C` signifie une régularisation plus faible, permettant au modèle de s'ajuster davantage aux données d'entraînement, ce qui peut potentiellement conduire à un surajustement. Une valeur plus faible favorise une régularisation plus forte, limitant la complexité du modèle pour éviter le surajustement.

2. **`kernel`** : Ce paramètre détermine le type de fonction noyau utilisée dans le modèle. `'linear'` pour un noyau linéaire, `'rbf'` (Radial basis function) pour un noyau gaussien adapté aux données non linéaires.

3. **`gamma`** : Le paramètre gamma influence la forme de la fonction noyau. Des valeurs plus élevées créent des frontières de décision plus complexes, adaptées aux données d'entraînement. Une valeur trop élevée peut entraîner un surajustement. Des valeurs plus faibles créent des frontières de décision plus simples.

4. **`coef0`** : Ce paramètre est utilisé par certains noyaux, comme le noyau polynomial ou sigmoïde. Il affecte la sensibilité du modèle aux coefficients.

En ajustant ces paramètres via une recherche exhaustive sur la grille, le modèle SVM peut être finement réglé pour s'adapter au mieux aux données spécifiques sur lesquelles il est entraîné.

Cette partie du code peut prendre un certain temps pour s'exécuter, donc nous l'avons exécutée une fois et nous avons ensuite commenté le code pour éviter de devoir l'exécuter à chaque fois.

In [ ]:

#param_grid_svm = {
 #   'C': [0.1, 1, 10],
 #  'kernel': ['linear', 'rbf'],
 #   'gamma': ['scale', 'auto', 0.0001,0.001,0.005,0.01,0.1, 1,10],
 #   'coef0': [0.0, 0.1, 0.5 ,0.558,1,5,10,50,100],
 #   'decision_function_shape': ['ovo', 'ovr']
#}

#svm_gridsearch = GridSearchCV(SVC(), param_grid_svm, n_jobs=12, cv=5, scoring='accuracy', verbose=100)

#svm_hyper_params_models = svm_gridsearch.fit(X_train, y_train)

#best_params_svm = svm_gridsearch.best_params_

#print(best_params_svm)


In [ ]:
best_params_svm = {'C': 1,
 'coef0': 0.0,
 'decision_function_shape': 'ovo',
 'gamma': 10,
 'kernel': 'rbf'}
best_params_svm

Initialisation du modéle

In [ ]:
svm = SVC(**best_params_svm)

Entrainer le modéle

In [ ]:
svm.fit(X_train, y_train)

On doit calculer plusieurs métriques d'évaluation de la performance du modèle :

1. **Précision** : La précision mesure le pourcentage de prédictions positives correctes parmi toutes les prédictions positives. Une précision élevée indique que le modèle a réussi à identifier avec précision les vrais positifs.

2. **Rappel** : Le rappel (ou sensibilité) mesure le pourcentage de vrais positifs identifiés par le modèle parmi tous les vrais positifs présents dans les données. Un rappel élevé signifie que le modèle a identifié la plupart des vrais positifs.

3. **Exactitude** : L'exactitude mesure le pourcentage total de prédictions correctes parmi toutes les prédictions. Elle indique la capacité globale du modèle à prédire correctement les classes.

4. **F1-score** : Le F1-score est la moyenne harmonique de la précision et du rappel. Il fournit un équilibre entre la précision et le rappel, offrant ainsi une mesure globale de la performance du modèle.

Ensuite, le code imprime ces métriques pour évaluer la qualité des prédictions du modèle SVM sur l'ensemble de données de test (`y_test`).

In [ ]:
y_pred = svm.predict(X_test)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print("Précision :", precision)
print("Rappel :", recall)
print("Exactitude :", accuracy)
print("F1-score :", f1)

Le code ci-dessous va tracer la matrice de confusion pour mieux visualiser nos resultats en montrant Tp:True postif, Fp: False postif , TN: True negatif , FN : false negatif

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
labels = ['survived', 'dead']
conf_matrix_df = pd.DataFrame(conf_matrix, index=labels, columns=labels)

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Réponses réelles')
plt.title('Matrice de Confusion')
plt.show()

Un recap de toutes les métriques

In [ ]:

cr = classification_report(y_test, y_pred)
print("Rapport de classification :\n", cr)


Ce code génère une courbe ROC (Receiver Operating Characteristic) pour évaluer la performance d'un modèle de classification binaire. Il calcule les taux de faux positifs (FPR) et de vrais positifs (TPR) à partir des données de test (y_test et y_pred), puis trace la courbe ROC avec l'aire sous la courbe (AUC) pour mesurer l'efficacité du modèle dans la discrimination entre les classes positives et négatives

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
# Tracer la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbe ROC')
plt.legend(loc='lower right')
plt.show()

Employez la bibliothèque Scikit-plot pour visualiser la courbe d'apprentissage du modèle.

In [ ]:
skplt.estimators.plot_learning_curve(svm, X_train, y_train)
plt.show()

### XGBoost

Xgboost est un modele d'apprentissage supervisé qui se base sur le boosting , L'idée principale derrière le boosting est de combiner plusieurs modèles d'apprentissage faibles pour créer un modèle fort.
Les paramètres initiaux comprennent des valeurs variées pour des hyperparamètres tels que le nombre d'estimateurs (n_estimators), le taux d'apprentissage (learning_rate), la profondeur maximale de l'arbre (max_depth), le poids minimal d'un enfant (min_child_weight), le terme de régularisation gamma (gamma), le taux de sous-échantillonnage des données (subsample), la fraction de caractéristiques à utiliser par arbre (colsample_bytree), l'alpha de régularisation L1 (reg_alpha), et l'alpha de régularisation L2 (reg_lambda).
On va mettre en place dans ce code une recherche exaustive qui évalue différentes combinaisons de ces paramètres pour trouver celle qui optimise la précision du modèle XGBoost.

Cette partie du code peut prendre un certain temps pour s'exécuter, donc nous l'avons exécutée une fois et avons ensuite commenté le code pour éviter de devoir l'exécuter à chaque fois.

In [ ]:
# # Nous définissons l'espace de recherche
# param_grid = {
#     'n_estimators': [100, 200, 300, 500],
#     'learning_rate': [0.01, 0.05, 0.1],
#     'max_depth': [3, 5, 7],
#     'min_child_weight': [1, 3, 5],
#     'gamma': [0.01, 0.05, 0.1],
#     'subsample': [0.7, 0.8, 0.9],
#     'colsample_bytree': [0.7, 0.8, 0.9],
#     'reg_alpha': [0, 0.01, 0.05],
#     'reg_lambda': [0, 0.01, 0.05],
# }

# # Puis nous créons l'objet GridSearchCV
# gridsearch = GridSearchCV(XGBClassifier(), param_grid, n_jobs=11, cv=5, scoring='accuracy', verbose=100)
# # Entraînez le modèle
# hyper_params_models = gridsearch.fit(X_train, y_train)

# # on enregistre les meilleurs paramètres
# best_params = gridsearch.best_params_

# # on affiche les meilleurs paramètres
# print(best_params)

Voici les meilleurs parmaetres qu'on a trouvé en utilisant GridSearch

In [ ]:
best_params = {'colsample_bytree': 0.7,
 'gamma': 0.05,
 'learning_rate': 0.1,
 'max_depth': 7,
 'min_child_weight': 1,
 'n_estimators': 500,
 'reg_alpha': 0.05,
 'reg_lambda': 0,
 'subsample': 0.8,
        
}


Instanciation et entrainement du modéle

In [ ]:
xgb =  XGBClassifier(**best_params,objective= 'binary:logistic',
 nthread=-1,

)
xgb.fit(X_train, y_train)

Evaluation du modele selon les metriques mentionnées dans la partie du SVM

In [ ]:
y_pred = xgb.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Précision :", precision)
print("Rappel :", recall)
print("Exactitude :", accuracy)
print("F1-score :", f1)

La matrice de confusion

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
labels = ['survived', 'dead']
conf_matrix_df = pd.DataFrame(conf_matrix, index=labels, columns=labels)

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Réponses réelles')
plt.title('Matrice de Confusion')
plt.show()

Tracer la courbe Roc en se servant des resultats de la matrice

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbe ROC')
plt.legend(loc='lower right')
plt.show()

Utilisez scikitplot pour tracer la courbe d'entraînement

In [ ]:

skplt.estimators.plot_learning_curve(xgb, X_train, y_train)
plt.show()

### Logistic Regression

La régression logistique est un algorithme de classification qui est utilisé pour résoudre des problèmes binaires (deux classes). Ses principaux paramètres dans ce contexte sont :

1. **`penalty`** : Cela spécifie la norme utilisée dans la régularisation. "l1" utilise la régularisation L1 (Lasso) qui ajoute la valeur absolue des coefficients, tandis que "l2" utilise la régularisation L2 (Ridge) qui ajoute le carré des coefficients.

2. **`C`** : Il contrôle l'inverse de la force de régularisation. Des valeurs plus petites spécifient une régularisation plus forte. La plage spécifiée ici (np.logspace(-4, 4, 10)) signifie que C variera de 0.0001 à 10000.

3. **`solver`** : C'est l'optimiseur utilisé dans le problème d'optimisation. "liblinear" est adapté aux petites données.

Le code utilise `GridSearchCV` pour rechercher les meilleures combinaisons de ces paramètres en termes de précision (scoring="accuracy") sur un ensemble de validation croisée de 5 plis (`cv=5`). Les meilleures valeurs de ces paramètres sont stockées dans `best_params_lgR` après l'exécution de la recherche sur la grille.

Cette partie du code peut prendre un certain temps pour s'exécuter, donc nous l'avons exécutée une fois et avons ensuite commenté le code pour éviter de devoir l'exécuter à chaque fois.

In [ ]:
# param_grid = {
#     "penalty": ["l1", "l2"],
#     "C": np.logspace(-4, 4, 10),
#      "solver": ["liblinear"]
# }

# grid_search = GridSearchCV(LogisticRegression(), param_grid, scoring="accuracy", cv=5, verbose=100)
# grid_search.fit(X_train, y_train)
# best_params_lgR = grid_search.best_params_

In [ ]:
best_params_lgR = {'C': 0.3593813663804626, 'penalty': 'l2', 'solver': 'liblinear'}

Entrainer le modéle

In [ ]:
lgr =  LogisticRegression(**best_params_lgR)
lgr.fit(X_train, y_train)

In [ ]:
y_pred = lgr.predict(X_test)

precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Précision :", precision)
print("Rappel :", recall)
print("Exactitude :", accuracy)
print("F1-score :", f1)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
labels = ['survived', 'dead']
conf_matrix_df = pd.DataFrame(conf_matrix, index=labels, columns=labels)

plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix_df, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Prédictions')
plt.ylabel('Réponses réelles')
plt.title('Matrice de Confusion')
plt.show()

In [ ]:
fpr, tpr, thresholds = roc_curve(y_test, y_pred)
roc_auc = auc(fpr, tpr)
# Tracer la courbe ROC
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, color='darkorange', lw=2, label='Courbe ROC (AUC = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlabel('Taux de faux positifs (FPR)')
plt.ylabel('Taux de vrais positifs (TPR)')
plt.title('Courbe ROC')
plt.legend(loc='lower right')
plt.show()

In [ ]:
skplt.estimators.plot_learning_curve(lgr, X_train, y_train)
plt.show()

### Comparaison entre nos trois modéles

In [ ]:

modeles = ['SVM', 'XGBOOST', 'LR']
precision = [0.8334228909188608, 0.912761355443403, 0.7839013778100072] 
rappel = [1.0, 0.816247582205029, 0.696969696969697] 
F1score = [0.9091441969519344, 0.8618107556160652, 0.7378839590443687]     
data = {'Modèle': modeles, 'Précision': precision, 'Rappel': rappel,'F1-Score' : F1score}
df = pd.DataFrame(data)
largeur_barre = 0.25
indices = np.arange(len(modeles))
plt.figure(figsize=(12, 8))
plt.bar(indices - largeur_barre, precision, largeur_barre, label='Précision', color='#B3E5FC')
plt.bar(indices + largeur_barre, rappel, largeur_barre, label='Rappel', color='#75E6DA')
plt.bar(indices , F1score, largeur_barre, label='f1-score', color='#CCD9E5')
plt.xlabel('Modèles')
plt.ylabel('Performances')
plt.title('Comparaison des Modèles (Précision ,Rappel et F1-Score)')
plt.xticks(indices, modeles)
plt.legend()
plt.show()

Apres le tracage du graphe ci-dessus nous allons intrepreter les resultats :

1. **SVM (Support Vector Machine)** :
   - **Précision :** 83.34%
   - **Rappel :** 100.0%
   - **F1-score :** 90.91%

   Le modèle SVM a une excellente sensibilité (rappel de 100%), ce qui signifie qu'il a réussi à identifier toutes les vraies valeurs positives dans le jeu de données. Cependant, cela vient au prix d'une précision légèrement inférieure, indiquant qu'il a également généré un certain nombre de faux positifs. Le score F1 de 90.91% équilibre précision et rappel, montrant une bonne performance globale.
2. **XGBoost** :
   - **Précision :** 91.28%
   - **Rappel :** 81.62%
   - **F1-score :** 86.18%

   XGBoost affiche la meilleure précision parmi les trois modèles, indiquant qu'il fait des prédictions positives correctes dans 91.28% des cas. Cependant, son rappel est légèrement inférieur, suggérant qu'il manque certaines vraies valeurs positives. Le score F1 de 86.18% est un bon compromis entre précision et rappel.
3. **Régression Logistique (LR)** :
   - **Précision :** 78.39%
   - **Rappel :** 69.70%
   - **F1-score :** 73.79%

   La régression logistique a des performances plus modestes en termes de précision et de rappel par rapport aux autres modèles. Cependant, elle offre un équilibre raisonnable entre précision et rappel, comme en témoigne son score F1 de 73.79%.


## Apprentissage non supervisée

### K-means

L'objectif principal de K-means est de diviser un ensemble de données en K clusters, où K est un nombre prédéfini. L'algorithme fonctionne en assignant chaque donnée au cluster dont le centroïde (le point central) est le plus proche en termes de distance euclidienne

Standarisation des données

In [ ]:
ct = ColumnTransformer([
    ('scale', StandardScaler(), cols_to_scale)
], remainder='passthrough')
X = ct.fit_transform(dfs)

Nous allons déterminer le nombre optimal de clusters pour nos données en utilisant deux méthodes différentes : la méthode du coude et la méthode de silhouette.

In [ ]:
inertias = []
for k in range(1, 11):  
    kmeans = KMeans(n_clusters=k,init = 'k-means++')
    kmeans.fit(X)  
    inertias.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(range(1, 11), inertias, marker='o', linestyle='--')
plt.xlabel('Nombre de clusters')
plt.ylabel('Inertie')
plt.title('Méthode du Coude pour trouver le meilleur K')
plt.show()

Pour le graphe ci-dessus, nous cherchons le point où l'ajout d'un nouveau cluster n'entraîne pas une réduction significative de la variation intra-cluster. En analysant le graphique, nous observons que la descente du graphe devient moins abrupte à partir du nombre de clusters égal à 3. Par conséquent, selon la méthode du coude, le meilleur nombre de clusters possible pour nos données est 3.

In [ ]:
silhouette_scores_df = pd.DataFrame(columns=['Nombre de Clusters', 'Silhouette Score'])

for k in range(2, 11):
    kmeans = KMeans(n_clusters=k)
    labels = kmeans.fit_predict(X)
    silhouette_avg = silhouette_score(X, labels)
    new_row = pd.DataFrame({'Nombre de Clusters': [k], 'Silhouette Score': [silhouette_avg]})
    silhouette_scores_df = pd.concat([silhouette_scores_df, new_row], ignore_index=True)
heatmap_data = silhouette_scores_df.pivot_table(index='Nombre de Clusters', values='Silhouette Score')

plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', fmt=".3f", linewidths=.5)
plt.title('Scores de Silhouette pour Différents Nombres de Clusters')
plt.show()


La methode de silhouette montre que le meuilleur nombre de cluster pour nos données c'est 2

Nous allons utiliser l'analyse en composantes principales (PCA) pour réduire la dimension de nos données et ainsi visualiser nos clusters de manière plus efficace.

K = 2

In [ ]:
pca_full = PCA()  
principal_components_full = pca_full.fit_transform(X)
kmeans = KMeans(n_clusters=2,init = 'k-means++') 
labels_2 = kmeans.fit_predict(principal_components_full)

plt.scatter(principal_components_full[:, 0], principal_components_full[:, 1], c=labels_2, cmap='viridis')
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')
plt.title('Visualisation des Clusters avec K-means (Caractéristiques d\'Origine)')
plt.show()

K = 3

In [ ]:
pca_full = PCA()  
principal_components_full = pca_full.fit_transform(X)
kmeans = KMeans(n_clusters=3,init = 'k-means++') 
labels_3 = kmeans.fit_predict(principal_components_full)
plt.scatter(principal_components_full[:, 0], principal_components_full[:, 1], c=labels_3, cmap='viridis')
plt.xlabel('Composante Principale 1')
plt.ylabel('Composante Principale 2')
plt.title('Visualisation des Clusters avec K-means (Caractéristiques d\'Origine)')
plt.show()

Les clusters sont bien determinés par les deux choix de nombres de clusters , Cependant on voit que y a un clustring un peu meilleur quand on utilise k= 2

Vu que notre probleme initial est un probléme de classification binaire nous allons tester le score 'jaccard_similarity' lorsque le k = 2 et nous comparons les labels de kmeans avec nos etiquettes principales

In [ ]:
jaccard_similarity = jaccard_score(y, labels_2, average='weighted')
print(f"Coefficient de Similarité de Jaccard : {jaccard_similarity}")

On remarque que le score de similarité est un peu faible , on déduit que nos données de base ne sont bien regroubé par l'algorithme k-means vu notre probléme principal 

### L'algorithme héarchique

L'algorithme hiérarchique est une méthode de regroupement non supervisée.Son objectif principal est de diviser un ensemble de données en clusters de manière progressive et itérative. L'algorithme commence par considérer chaque point de données comme un cluster individuel, puis fusionne progressivement les clusters voisins en fonction de leur similarité, formant ainsi des clusters plus grands.

Contrairement à certains autres algorithmes de clustering, l'algorithme hiérarchique ne nécessite pas de spécifier le nombre de clusters à l'avance. Il permet d'explorer différentes configurations de clusters en fusionnant progressivement les clusters voisins à divers niveaux

Le paramètre initial clé pour optimiser l'algorithme hiérarchique est la méthode de liaison (linkage method). La méthode de liaison détermine comment mesurer la distance ou la similarité entre les clusters lors de leur fusion

Pour determiner le meilleur parametre pour optimiser cet algorithme , nous allons implémenter un algorithme qui va detrminer la meilleur méthode possible pour le linkage et la methode silhouette pour s'assurer du résultat.

In [ ]:
pca_full = PCA()  
principal_components_full = pca_full.fit_transform(X)
linkage_methods = ['ward', 'complete', 'average', 'single']

plt.figure(figsize=(12, 8))
for i, linkage_method in enumerate(linkage_methods, 1):
    model = AgglomerativeClustering(linkage=linkage_method)
    labels = model.fit_predict(principal_components_full)
    plt.subplot(2, 2, i)
    plt.scatter(principal_components_full[:, 0], principal_components_full[:, 1], c=labels, cmap='viridis')
    plt.title(f'Linkage Method: {linkage_method.capitalize()}')
    
plt.tight_layout()
plt.show()

D'apres les resultats ci-dessus la meilleur methode de linkage à utiliser pour bien regrouper notre données c'est la méthode : ward

In [ ]:
silhouette_scores_df_hier = pd.DataFrame(columns=['Nombre de Clusters', 'Silhouette Score'])

for k in range(2, 11):
    model = AgglomerativeClustering(n_clusters=k) 
    labels = model.fit_predict(X)
    silhouette_avg = silhouette_score(X, labels)
    new_row = pd.DataFrame({'Nombre de Clusters': [k], 'Silhouette Score': [silhouette_avg]})
    silhouette_scores_df_hier = pd.concat([silhouette_scores_df_hier, new_row], ignore_index=True)
heatmap_data = silhouette_scores_df_hier.pivot_table(index='Nombre de Clusters', values='Silhouette Score')

plt.figure(figsize=(8, 6))
sns.heatmap(heatmap_data, annot=True, cmap='YlGnBu', fmt=".3f", linewidths=.5)
plt.title('Scores de Silhouette pour Différents Nombres de Clusters')
plt.show()



Comme montré ci-dessus le meilleur nombre de cluster possible c'est lorsque k=2 , et ça correspond trés bien au résultat trouvé par l'algorithme ci-dessus.

In [ ]:
model = AgglomerativeClustering() 
cluster_labels = model.fit_predict(X)
jaccard_similarity = jaccard_score(y, labels, average='weighted')
print(f"Coefficient de Similarité de Jaccard : {jaccard_similarity}")

Le score est faible par rapport à l'algorithme k-means